In [1]:
%%capture
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


## Data Pre-processing

In [3]:
from datasets import load_dataset, load_metric, Audio

# takes a few mins (maybe 25-30)
ds_orig = load_dataset("edinburghcstr/ami", "ihm")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset ami downloaded and prepared to /root/.cache/huggingface/datasets/edinburghcstr___ami/ihm/0.0.0/0d128d0aa8145d0f16f3d5b4da86c5d5759dbe9e8f947fda04b25edb56442bd5. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
COLUMNS_TO_KEEP = ['text', 'audio']
all_columns = ds_orig["train"].column_names
columns_to_remove = set(all_columns) - set(COLUMNS_TO_KEEP)
ds = ds_orig.remove_columns(columns_to_remove)

In [5]:
ds_train = ds["train"]
ds_val = ds["validation"]
ds_test = ds["test"]

In [6]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(ds_train.remove_columns(["audio"]), num_examples=10)

,text
0,FOR BETTER WANT OF A BETTER WORD YOU KNOW
1,GO
2,OH I'LL GO FOR A ONE
3,NOT N SO ENOUGH UH WELL YEAH YEAH
4,NO YOU CAN'T
5,THE FANCY COLOURS
6,YEAH YOU YO YEAH BUT MOST OFTEN NOT
7,YEAH
8,UH-HUH HUH-HUH
9,NO


In [8]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [9]:
ds_train = ds_train.map(remove_special_characters)
ds_val = ds_val.map(remove_special_characters)
ds_test = ds_test.map(remove_special_characters)

Map:   0%|          | 0/108502 [00:00<?, ? examples/s]

Map:   0%|          | 0/13098 [00:00<?, ? examples/s]

Map:   0%|          | 0/12643 [00:00<?, ? examples/s]

In [10]:
show_random_elements(ds_train.remove_columns(["audio"]), num_examples=10)

,text
0,just through uh the easy uh design
1,yeah but
2,but i don't think it should be very it should be one of the big buttons for example
3,yeah yeah
4,but
5,yeah it's still erasing
6,yes
7,as you can see w opening acquaintance tool training project plan discussion and closing
8,means equivalent widths yeah
9,somehow


In [11]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [12]:
vocab_train = ds_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=ds_train.column_names)
vocab_val = ds_val.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=ds_val.column_names)
vocab_test = ds_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=ds_test.column_names)

Map:   0%|          | 0/108502 [00:00<?, ? examples/s]

Map:   0%|          | 0/13098 [00:00<?, ? examples/s]

Map:   0%|          | 0/12643 [00:00<?, ? examples/s]

In [13]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_val["vocab"][0]) | set(vocab_test["vocab"][0]))

In [14]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'l': 0,
 'v': 1,
 '*': 2,
 'r': 3,
 'x': 4,
 'z': 5,
 'o': 6,
 't': 7,
 'e': 8,
 'p': 9,
 ' ': 10,
 'n': 11,
 'k': 12,
 'f': 13,
 "'": 14,
 'q': 15,
 'w': 16,
 'm': 17,
 'd': 18,
 'u': 19,
 'a': 20,
 'b': 21,
 'i': 22,
 'c': 23,
 '@': 24,
 'j': 25,
 'h': 26,
 's': 27,
 'y': 28,
 'g': 29}

In [15]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [16]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

32

In [17]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [18]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [19]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [20]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [21]:
ds_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/b093dd855ad7f053bb12157f9ce4b51aaeaf29ccaf78fdc8150b3a8125130e2b/EN2001a/train_ami_en2001a_h04_meo069_0330297_0330718.wav',
 'array': array([ 0.00231934, -0.00183105, -0.00543213, ..., -0.00238037,
        -0.00244141, -0.00219727], dtype=float32),
 'sampling_rate': 16000}

In [22]:
ds_train = ds_train.cast_column("audio", Audio(sampling_rate=16_000))
ds_val = ds_val.cast_column("audio", Audio(sampling_rate=16_000))
ds_test = ds_test.cast_column("audio", Audio(sampling_rate=16_000))

In [23]:
ds_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/b093dd855ad7f053bb12157f9ce4b51aaeaf29ccaf78fdc8150b3a8125130e2b/EN2001a/train_ami_en2001a_h04_meo069_0330297_0330718.wav',
 'array': array([ 0.00231934, -0.00183105, -0.00543213, ..., -0.00238037,
        -0.00244141, -0.00219727], dtype=float32),
 'sampling_rate': 16000}

In [24]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(ds_train)-1)

ipd.Audio(data=ds_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

In [25]:
rand_int = random.randint(0, len(ds_train)-1)

print("Target text:", ds_train[rand_int]["text"])
print("Input array shape:", ds_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", ds_train[rand_int]["audio"]["sampling_rate"])

Target text: they're not as prominent as the main features 
Input array shape: (29600,)
Sampling rate: 16000


In [26]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [27]:
ds_train = ds_train.map(prepare_dataset, remove_columns=ds_train.column_names, num_proc=4)
ds_val = ds_val.map(prepare_dataset, remove_columns=ds_val.column_names, num_proc=4)
ds_test = ds_test.map(prepare_dataset, remove_columns=ds_test.column_names, num_proc=4)

Map (num_proc=4):   0%|          | 0/108502 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__`

Map (num_proc=4):   0%|          | 0/13098 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__`

Map (num_proc=4):   0%|          | 0/12643 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__`

## Training

In [43]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [44]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [45]:
wer_metric = load_metric("wer")

<ipython-input-45-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [46]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [47]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-robust-ft-swbd-300h", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--wav2vec2-large-robust-ft-swbd-300h/snapshots/828a8386883f64170e04fae06dd866e0fe97de6b/config.json
Model config Wav2Vec2Config {
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extr

In [48]:
model.freeze_feature_extractor()

/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1639: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [49]:
model.gradient_checkpointing_enable()

In [50]:
from transformers import TrainingArguments

# just training on cpu so not in fp16
training_args = TrainingArguments(
  output_dir="/content/gdrive/MyDrive/wav2vec2-large-robust-ft-swbd-300h-demo",
  #output_dir="./wav2vec2-large-robust-ft-swbd-300h-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  #fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [51]:
ds_train

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 108502
})

In [52]:
from transformers import Trainer

# for now just select a small amount of data so it trains fast...

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=ds_train.select([0]),
    eval_dataset=ds_test.select([0]),
    tokenizer=processor.feature_extractor,
)

In [53]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 30
  Number of trainable parameters = 311261344
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=30, training_loss=3.9849684397379557, metrics={'train_runtime': 509.728, 'train_samples_per_second': 0.059, 'train_steps_per_second': 0.059, 'total_flos': 3825029085696000.0, 'train_loss': 3.9849684397379557, 'epoch': 30.0})

In [54]:
trainer.save_model("/content/gdrive/MyDrive/wav2vec2-large-robust-ft-swbd-300h-model")

Saving model checkpoint to /content/gdrive/MyDrive/wav2vec2-large-robust-ft-swbd-300h-model
Configuration saved in /content/gdrive/MyDrive/wav2vec2-large-robust-ft-swbd-300h-model/config.json
Model weights saved in /content/gdrive/MyDrive/wav2vec2-large-robust-ft-swbd-300h-model/pytorch_model.bin
Feature extractor saved in /content/gdrive/MyDrive/wav2vec2-large-robust-ft-swbd-300h-model/preprocessor_config.json


## Evaluation

In [36]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Trainer
import torch

In [51]:
#model = Wav2Vec2ForCTC.from_pretrained("/content/gdrive/MyDrive/wav2vec2-large-robust-ft-swbd-300h-model")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-robust-ft-swbd-300h")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--wav2vec2-large-robust-ft-swbd-300h/snapshots/828a8386883f64170e04fae06dd866e0fe97de6b/config.json
Model config Wav2Vec2Config {
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extra

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--facebook--wav2vec2-large-robust-ft-swbd-300h/snapshots/828a8386883f64170e04fae06dd866e0fe97de6b/pytorch_model.bin
All model checkpoint weights were used when initializing Wav2Vec2ForCTC.

All the weights of Wav2Vec2ForCTC were initialized from the model checkpoint at facebook/wav2vec2-large-robust-ft-swbd-300h.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Wav2Vec2ForCTC for predictions without further training.


In [52]:
trainer = Trainer(model=model)
# how to make an individual prediction
y = trainer.predict(ds_val.select([0]))

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 1
  Batch size = 8


In [53]:
y

PredictionOutput(predictions=array([[[  9.115491 , -15.125415 , -15.891221 , ...,  -5.8609495,
          -5.08603  ,  -5.8642416],
        [  4.0983934, -15.8095455, -16.274248 , ...,  -4.6487894,
          -2.8354423,  -5.5427504],
        [  6.679947 , -17.486929 , -17.731813 , ...,  -6.1438975,
          -4.0898743,  -4.0304394],
        ...,
        [  9.724043 , -14.817215 , -15.28695  , ...,  -4.7162323,
          -5.778461 ,  -4.88419  ],
        [  9.2227955, -14.379663 , -14.86626  , ...,  -5.0055943,
          -5.9026537,  -4.381193 ],
        [  1.8667985, -13.882484 , -13.887471 , ...,  -5.919756 ,
          -4.007285 ,  -7.1707277]]], dtype=float32), label_ids=array([[21, 19,  7, 10,  0, 22, 12,  8, 10, 17,  6, 21, 22,  0,  8, 10,
         9, 26,  6, 11,  8, 27, 10, 26, 20,  1,  8, 10, 27, 23,  3,  8,
         8, 11, 27, 10, 20, 11, 18, 10,  7, 26,  8, 28, 14,  3,  8, 10,
        23, 26,  8, 20,  9, 10]]), metrics={'test_loss': 492.0483093261719, 'test_runtime': 3.2852, 't

In [54]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"]).unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [55]:
#results = ds_val.map(map_to_result, remove_columns=ds_val.column_names)

# just do some eval to get an idea
results = ds_val.select(list(range(20))).map(map_to_result, remove_columns=ds_val.column_names)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [56]:
wer_metric = load_metric("wer")

In [57]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 1.006


In [58]:
# TODO: pretty sure its not my model thats messed up since I loaded the baseline one and it doesn't work here either
show_random_elements(results)

,pred_str,text
0,lx,yeah
1,l@woxl'lpjlzlxultl@lpl'zlxiltl lqlklxneslzlxlkl[UNK]lwflzlzl lkxle qlxonzlhlfzlxlanlzlelclx,but like mobile phones have screens and they're cheap
2,ltlx,right
3,lktlxmtwxkntw'qxneszxlelxale'l'lx@lwolotl lxt lxmtwfxolzl'lzlslplkpt lxotx@zxe@'zxotlxlilpl qlxmtwfxlutlozlxat lolflt'lx,so you should have a call button on your television to be able to find your remote control
4,lpxuze lxpolxywkloxkzlzulkx'pjzlxmzelnlx,i mean it just seems like yeah
5,lmzelnlxonzmhfzxkl'zleljxe qxonzmx'tltjxlp lolz'l'lplblz lolxle qlxpxqt hoxj tdlxlonzlxpxbwzklkxlpohkxonzxdnt'zx'pljzlx@lpl ltlalwl'leflxslplkptl lxonp blx,yeah they're sleek and they look intelligent and i don't know they're i guess it's the whole like binocular vision thing
6,lilwflmlx,furry
7,l@lwolxlpohkxltl l'mlxelxcl'lelklolplalxonlpl lblxlflzlel'l'lmlxonlzxonp lbxpl xonzlxdlelx,but it's only a plastic thing r really the thing on the wall
8,lplilxdlzlhlxlel'l'lxlolflel'lxltwlflxilelstfplozxlel lplulel'lxloltlxkltluzlxwclxonzlxanelfealolzflplklolplalklxltilxonelolxlel lpule'lx,if we'll all draw our favourite animal to sum up the characteristics of that animal
9,lwlx,mm


In [59]:
with torch.no_grad():
  logits = model(torch.tensor(ds_val[:1]["input_values"])).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

"l @ w w o o x x l ' l p j l z l x x u u l t l l l l @ l l p l l ' ' z l x x i l l l t l l l | l q l k l x x x n e e s l z l l x x l k l l [UNK] l w f l z l l z l l | l l l k k x l e | | q l x o n n z l l l h l f z z l x x l a n n l l z l e l l l l c l l l l l l l l l x"

## References
* https://huggingface.co/datasets/edinburghcstr/ami
* https://huggingface.co/facebook/wav2vec2-large-robust-ft-swbd-300h
* https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLSR_Wav2Vec2_on_Turkish_ASR_with_%F0%9F%A4%97_Transformers.ipynb#scrollTo=72737oog2F6U
* https://github.com/huggingface/blog/blob/main/audio-datasets.md
* https://huggingface.co/blog/fine-tune-wav2vec2-english